In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/')

In [2]:
import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools
import os
from pathlib import Path

from model.GRU_model import *
from pipeline_config import *
from utils import *

In [3]:
# Seeds for comparisons:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)
torch.use_deterministic_algorithms(True)


In [4]:
with open(train_dataset_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)
    
with open(test_dataset_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

with open(parent_to_son_list_path, 'rb') as f:
    parent_to_son_list_dict = pickle.load(f)

with open(hgru_model_weights_path, 'rb') as f:
    hgru_weight_dict = pickle.load(f)

with open(coefficient_dict_path, 'rb') as f:
    coefficient_dict = pickle.load(f)

In [5]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs: {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.1
Epochs: 100


In [6]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        if category == 'Food and beverages':
            continue
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model =GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
        basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [7]:
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

# Loss Analysis 1:

No Loss Term 1:

In [8]:
# Sanity Check:
weightspath_1

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_1/'

In [9]:
path = os.getcwd()
desired_path = path + '/models_weights_1/'
desired_path

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_1/'

In [10]:
def bi_directional_model_1(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict,loss_coef_1, loss_coef_2, loss_coef_3, alpha):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights_1/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 0
    loss_coef_2= 1*np.exp(-10)
    loss_coef_3= 0.0001
    alpha= 1.0

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)

In [11]:
bidrectional_models_1 = bi_directional_model_1(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath_1, hgru_weight_dict, coefficient_dict, 0, loss_coef_2, loss_coef_3, alpha)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_1/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_1/
num categories: 1
category id|name: 45|Accommodation services
------------------------------------------------------------------
num categories: 2
category id|name: 19|Actual rentals for housing
------------------------------------------------------------------
num categories: 3
category id|name: 15|Alcoholic beverages
------------------------------------------------------------------
num categories: 4
category id|name: 38|Audio-visual, photographic and information processing equipment
------------------------------------------------------------------
num categories: 5
category id|name: 17|Clothing
------------------------------------------------------------------
num categories: 6
category id|name: 23|Electricity, g

Save prediction dict:

In [12]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [13]:
predictions_dict_1 = get_results_on_test_set(weightspath_1, train_dataset_dict, test_dataset_dict, categories = categories)
with open(test_predictions_path_1, 'wb') as handle:
    pickle.dump(predictions_dict_1, handle, protocol=pickle.HIGHEST_PROTOCOL)

Accommodation services
Actual rentals for housing
Alcoholic beverages
Audio-visual, photographic and information processing equipment
Clothing
Electricity, gas and other fuels
Financial services n.e.c.
Food
Footwear
Furniture and furnishings, carpets and other floor coverings
Glassware, tableware and household utensils
Goods and services for routine household maintenance
Household appliances
Household textiles
Imputed rentals for housing
Insurance
Maintenance and repair of the dwelling
Medical products, appliances and equipment
Newspapers, books and stationery
Non-alcoholic beverages
Operation of personal transport equipment
Other major durables for recreation and culture
Other recreational items and equipment, gardens and pets
Other services n.e.c.
Out-patient services
Package holidays
Personal care
Personal effects n.e.c.
Postal services
Purchase of vehicles
Recreational and cultural services
Restaurant services
Social protection
Telephone equipment
Telephone services
Tobacco
Tools a

# Loss Analysis 2:

No Loss Term 2:

In [14]:
# Sanity Check:
weightspath_2

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_2/'

In [15]:
def bi_directional_model_2(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict,loss_coef_1, loss_coef_2, loss_coef_3, alpha):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights_2/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 1*np.exp(-10)
    loss_coef_2= 0
    loss_coef_3= 0.0001

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)

In [16]:
bidrectional_models_2 = bi_directional_model_2(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath_2, hgru_weight_dict, coefficient_dict, loss_coef_1, 0, loss_coef_3, alpha)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_2/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_2/
num categories: 1
category id|name: 45|Accommodation services
------------------------------------------------------------------
num categories: 2
category id|name: 19|Actual rentals for housing
------------------------------------------------------------------
num categories: 3
category id|name: 15|Alcoholic beverages
------------------------------------------------------------------
num categories: 4
category id|name: 38|Audio-visual, photographic and information processing equipment
------------------------------------------------------------------
num categories: 5
category id|name: 17|Clothing
------------------------------------------------------------------
num categories: 6
category id|name: 23|Electricity, g

In [17]:
predictions_dict_2 = get_results_on_test_set(weightspath_2, train_dataset_dict, test_dataset_dict, categories = categories)
with open(test_predictions_path_2, 'wb') as handle:
    pickle.dump(predictions_dict_2, handle, protocol=pickle.HIGHEST_PROTOCOL)

Accommodation services
Actual rentals for housing
Alcoholic beverages
Audio-visual, photographic and information processing equipment
Clothing
Electricity, gas and other fuels
Financial services n.e.c.
Food
Footwear
Furniture and furnishings, carpets and other floor coverings
Glassware, tableware and household utensils
Goods and services for routine household maintenance
Household appliances
Household textiles
Imputed rentals for housing
Insurance
Maintenance and repair of the dwelling
Medical products, appliances and equipment
Newspapers, books and stationery
Non-alcoholic beverages
Operation of personal transport equipment
Other major durables for recreation and culture
Other recreational items and equipment, gardens and pets
Other services n.e.c.
Out-patient services
Package holidays
Personal care
Personal effects n.e.c.
Postal services
Purchase of vehicles
Recreational and cultural services
Restaurant services
Social protection
Telephone equipment
Telephone services
Tobacco
Tools a

# Loss Analysis 3:

No Loss Term 3:

In [18]:
weightspath_3

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_3/'

In [19]:
def bi_directional_model_3(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict,loss_coef_1, loss_coef_2, loss_coef_3, alpha):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights_3/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 1*np.exp(-10)
    loss_coef_2= 1*np.exp(-10)
    loss_coef_3= 0
    
    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)

In [20]:
bidrectional_models_3 = bi_directional_model_3(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath_3, hgru_weight_dict, coefficient_dict, loss_coef_1, loss_coef_2, 0, alpha)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_3/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_3/
num categories: 1
category id|name: 45|Accommodation services
------------------------------------------------------------------
num categories: 2
category id|name: 19|Actual rentals for housing
------------------------------------------------------------------
num categories: 3
category id|name: 15|Alcoholic beverages
------------------------------------------------------------------
num categories: 4
category id|name: 38|Audio-visual, photographic and information processing equipment
------------------------------------------------------------------
num categories: 5
category id|name: 17|Clothing
------------------------------------------------------------------
num categories: 6
category id|name: 23|Electricity, g

In [21]:
predictions_dict_3 = get_results_on_test_set(weightspath_3, train_dataset_dict, test_dataset_dict, categories = categories)
with open(test_predictions_path_3, 'wb') as handle:
    pickle.dump(predictions_dict_3, handle, protocol=pickle.HIGHEST_PROTOCOL)

Accommodation services
Actual rentals for housing
Alcoholic beverages
Audio-visual, photographic and information processing equipment
Clothing
Electricity, gas and other fuels
Financial services n.e.c.
Food
Footwear
Furniture and furnishings, carpets and other floor coverings
Glassware, tableware and household utensils
Goods and services for routine household maintenance
Household appliances
Household textiles
Imputed rentals for housing
Insurance
Maintenance and repair of the dwelling
Medical products, appliances and equipment
Newspapers, books and stationery
Non-alcoholic beverages
Operation of personal transport equipment
Other major durables for recreation and culture
Other recreational items and equipment, gardens and pets
Other services n.e.c.
Out-patient services
Package holidays
Personal care
Personal effects n.e.c.
Postal services
Purchase of vehicles
Recreational and cultural services
Restaurant services
Social protection
Telephone equipment
Telephone services
Tobacco
Tools a

# Loss Analysis 1+2

In [22]:
def bi_directional_model_1_2(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict,loss_coef_1, loss_coef_2, loss_coef_3, alpha):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights_2/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 0
    loss_coef_2= 0
    loss_coef_3= 0.0001

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)

In [23]:
bi_directional_model_1_2 = bi_directional_model_1_2(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath_1_2, hgru_weight_dict, coefficient_dict, 0, 0, loss_coef_3, alpha)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_2/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/bidirectional/models_weights_2/
num categories: 1
category id|name: 45|Accommodation services
------------------------------------------------------------------
num categories: 2
category id|name: 19|Actual rentals for housing
------------------------------------------------------------------
num categories: 3
category id|name: 15|Alcoholic beverages
------------------------------------------------------------------
num categories: 4
category id|name: 38|Audio-visual, photographic and information processing equipment
------------------------------------------------------------------
num categories: 5
category id|name: 17|Clothing
------------------------------------------------------------------
num categories: 6
category id|name: 23|Electricity, g

In [24]:
predictions_dict_1_2 = get_results_on_test_set(weightspath_1_2, train_dataset_dict, test_dataset_dict, categories = categories)
with open(test_predictions_path_1_2, 'wb') as handle:
    pickle.dump(predictions_dict_1_2, handle, protocol=pickle.HIGHEST_PROTOCOL)

Accommodation services
Actual rentals for housing
Alcoholic beverages
Audio-visual, photographic and information processing equipment
Clothing
Electricity, gas and other fuels
Financial services n.e.c.
Food
Footwear
Furniture and furnishings, carpets and other floor coverings
Glassware, tableware and household utensils
Goods and services for routine household maintenance
Household appliances
Household textiles
Imputed rentals for housing
Insurance
Maintenance and repair of the dwelling
Medical products, appliances and equipment
Newspapers, books and stationery
Non-alcoholic beverages
Operation of personal transport equipment
Other major durables for recreation and culture
Other recreational items and equipment, gardens and pets
Other services n.e.c.
Out-patient services
Package holidays
Personal care
Personal effects n.e.c.
Postal services
Purchase of vehicles
Recreational and cultural services
Restaurant services
Social protection
Telephone equipment
Telephone services
Tobacco
Tools a

In [25]:
# Sanity Tests:

In [26]:
predictions_dict_1['Food']

tensor([[ 1.9565,  0.1465,  2.2862, -1.4571, -0.0629,  2.1968, -1.6417,  0.0498,
         -0.4452, -0.1938, -1.4162,  0.3958,  3.0494, -1.0778,  0.6604, -0.2367,
          0.2681,  1.9547, -1.8108,  0.2687, -1.0220, -0.4843, -0.8463,  1.1873,
          2.6713, -2.1568,  1.1449,  0.6305]])

In [27]:
predictions_dict_2['Food']

tensor([[ 3.4667, -0.5328,  1.5012, -0.6985, -0.1365,  2.2750, -1.3498, -0.4775,
          0.0397, -0.0572, -2.4118,  0.7006,  3.2241, -0.8774,  0.1415, -0.2747,
          0.1124,  2.4366, -1.2814, -0.8852, -0.9093, -0.3248, -0.8964,  1.6280,
          3.0215, -1.4367,  0.1733,  0.7555]])

In [28]:
predictions_dict_3['Food']

tensor([[ 1.8132,  0.0547,  1.3841, -0.1048,  0.2976,  2.3092, -0.7548, -0.4424,
         -0.8744,  0.2049, -2.1023,  0.5649,  3.4522, -0.9312,  0.2670, -0.1493,
         -0.2305,  2.0821, -0.9054, -0.1862, -0.1639,  0.3851, -1.4192,  1.6084,
          3.5690, -1.0557,  1.2844,  0.1434]])

In [29]:
predictions_dict_1_2['Food']

tensor([[ 1.9565,  0.1465,  2.2862, -1.4571, -0.0629,  2.1968, -1.6417,  0.0498,
         -0.4452, -0.1938, -1.4162,  0.3958,  3.0494, -1.0778,  0.6604, -0.2367,
          0.2681,  1.9547, -1.8108,  0.2687, -1.0220, -0.4843, -0.8463,  1.1873,
          2.6713, -2.1568,  1.1449,  0.6305]])